# 時中指數

# Load Data 

In [1]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')

In [53]:
df.head(3)

,item_id,title,category,content,link,date,photo_link,tokens_v2,top_keys_freq,summary,sentiment
0,aipl_20220124_1001,殲16D首擾台 專家示警：國軍應強化電子作戰,政治,中共殲16D新型電戰機今天首次擾台，學者及退將分析，殲16D可能已量產並進行實戰化運用，國軍...,https://www.cna.com.tw/news/aipl/202201240357....,2022-01-24,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,"['中共', '新型', '電戰機', '學者', '退將', '分析', '量產', '進...","[('中共', 7), ('台灣', 7), ('電子', 6), ('美軍', 6), (...","['其中殲16D新型電戰機為首次現蹤', '中共殲16D新型電戰機今天首次擾台', '13架...",0.96
1,aipl_20220124_1002,朝野協商總預算在促轉會卡關 25日拚三讀添變數,政治,立法院朝野黨團今天繼續協商總預算案，不過到促轉會的預算提案時，朝野為了蔣經國總統圖書館議題起...,https://www.cna.com.tw/news/aipl/202201240390....,2022-01-24,NaN,"['立法院', '朝野', '黨團', '總預算案', '促轉會', '預算', '提案',...","[('預算', 7), ('提案', 7), ('黨團', 6), ('總統', 6), (...","['民進黨立法院黨團總召柯建銘說', '促轉會說這叫威權', '不過到促轉會的預算提案時',...",0.74
2,aipl_20220124_1003,朱立倫批修法打假球 民進黨：雙管齊下遏阻酒駕,政治,立法院今天三讀修正通過加重酒駕刑罰及行政罰的相關條文，國民黨主席朱立倫表示，酒駕零容忍，民進...,https://www.cna.com.tw/news/aipl/202201240374....,2022-01-24,NaN,"['立法院', '三讀', '修正', '加重', '酒駕', '刑罰', '行政罰', '...","[('酒駕', 16), ('國民黨', 8), ('立法院', 4), ('修正', 4)...","['立法院臨時會院會今天三讀修正通過加重酒駕刑罰及行政罰的相關條文', '今天酒駕罰則加重的...",0.13


In [3]:
df.shape

(26179, 11)

# Filter news for selected keywords

In [4]:
import pandas as pd
from datetime import datetime, timedelta
import re

# # Load Data 
# df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')
# Step 0: Filter news articles using the following function
# Searching keywords from "content" column
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [5]:
query_keywords = ['陳時中','時中阿伯'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)

In [6]:
len(df_query)

368

# Count how many pieces of news containing these keywords計算各類別多少篇文章提到該關鍵字

# Count how many times these keywords were mentioned in each category計算各類別出現關鍵字次數

In [7]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']

def count_keyword(df_query, query_keywords):

    cate_occurrence = {}
    cate_freq = {}
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0
        

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.category] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        # 計算這一篇文章的content中重複含有多少個這些關鍵字(頻率)
        freq = sum([ len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']
    return cate_freq, cate_occurrence, total_articles, total_frequency


In [8]:
query_keywords = ['陳時中','時中阿伯'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'政治': 153,
  '科技': 0,
  '運動': 5,
  '證卷': 1,
  '產經': 4,
  '娛樂': 1,
  '生活': 1117,
  '國際': 24,
  '社會': 0,
  '文化': 1,
  '兩岸': 0,
  '全部': 1306},
 {'政治': 49,
  '科技': 0,
  '運動': 2,
  '證卷': 1,
  '產經': 3,
  '娛樂': 1,
  '生活': 294,
  '國際': 17,
  '社會': 0,
  '文化': 1,
  '兩岸': 0,
  '全部': 368},
 368,
 1306)

In [9]:
query_keywords = ['陳時中','時中'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'政治': 308,
  '科技': 0,
  '運動': 10,
  '證卷': 2,
  '產經': 9,
  '娛樂': 5,
  '生活': 2241,
  '國際': 52,
  '社會': 1,
  '文化': 2,
  '兩岸': 3,
  '全部': 2633},
 {'政治': 51,
  '科技': 0,
  '運動': 2,
  '證卷': 1,
  '產經': 4,
  '娛樂': 3,
  '生活': 300,
  '國際': 21,
  '社會': 1,
  '文化': 1,
  '兩岸': 3,
  '全部': 387},
 387,
 2633)

In [10]:
import re

In [11]:
content="陳時中,....，陳時中00，陳時中說，"
keyword="陳時中"
re.findall(keyword, content)

['陳時中', '陳時中', '陳時中']

In [12]:
len(re.findall(keyword, content))

3

In [13]:
keyword="時中"
re.findall(keyword, content)

['時中', '時中', '時中']

In [14]:
keyword="陳中"
re.findall(keyword, content)

[]

# Caclulate date-based reported frequency of these keywords計算被報導的次數以時間為基礎

In [15]:
def get_keyword_occurrence_time_series(df_query):
    date_samples = df_query.date
    query_freq = pd.DataFrame({'date_index': pd.to_datetime(date_samples), 'freq': [1 for _ in range(len(df_query))]})
    data = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()
    line_xy_data = []
    for i, idx in enumerate(data.index):
        row = {'x': idx.strftime('%Y-%m-%d'), 'y': int(data.iloc[i].freq)}
        line_xy_data.append(row)
    return line_xy_data

In [16]:
get_keyword_occurrence_time_series(df_query)

[{'x': '2022-03-26', 'y': 6},
 {'x': '2022-03-27', 'y': 11},
 {'x': '2022-03-28', 'y': 10},
 {'x': '2022-03-29', 'y': 7},
 {'x': '2022-03-30', 'y': 17},
 {'x': '2022-03-31', 'y': 9},
 {'x': '2022-04-01', 'y': 12},
 {'x': '2022-04-02', 'y': 11},
 {'x': '2022-04-03', 'y': 9},
 {'x': '2022-04-04', 'y': 11},
 {'x': '2022-04-05', 'y': 12},
 {'x': '2022-04-06', 'y': 17},
 {'x': '2022-04-07', 'y': 19},
 {'x': '2022-04-08', 'y': 13},
 {'x': '2022-04-09', 'y': 16},
 {'x': '2022-04-10', 'y': 8},
 {'x': '2022-04-11', 'y': 9},
 {'x': '2022-04-12', 'y': 16},
 {'x': '2022-04-13', 'y': 19},
 {'x': '2022-04-14', 'y': 20},
 {'x': '2022-04-15', 'y': 16},
 {'x': '2022-04-16', 'y': 12},
 {'x': '2022-04-17', 'y': 12},
 {'x': '2022-04-18', 'y': 17},
 {'x': '2022-04-19', 'y': 14},
 {'x': '2022-04-20', 'y': 13},
 {'x': '2022-04-21', 'y': 15},
 {'x': '2022-04-22', 'y': 18},
 {'x': '2022-04-23', 'y': 18}]

# Put them together

In [17]:

query_keywords = ['陳時中','時中阿伯'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'

df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
freqByDate = get_keyword_occurrence_time_series(df_query)


selectedCategories = ['全部', '政治', '產經', '生活', '社會']
# selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

freqByCate = [cate_occurrence[k] for k in selectedCategories]

response =  {'freqByDate': freqByDate,
           'freqByCate': freqByCate,
           'category': selectedCategories,
           'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
           'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
           }

In [18]:
freqByCate

[368, 49, 3, 294, 0]

In [19]:
cate_freq

{'政治': 153,
 '科技': 0,
 '運動': 5,
 '證卷': 1,
 '產經': 4,
 '娛樂': 1,
 '生活': 1117,
 '國際': 24,
 '社會': 0,
 '文化': 1,
 '兩岸': 0,
 '全部': 1306}

In [20]:
cate_occurrence

{'政治': 49,
 '科技': 0,
 '運動': 2,
 '證卷': 1,
 '產經': 3,
 '娛樂': 1,
 '生活': 294,
 '國際': 17,
 '社會': 0,
 '文化': 1,
 '兩岸': 0,
 '全部': 368}

In [21]:
freqByDate

[{'x': '2022-03-26', 'y': 6},
 {'x': '2022-03-27', 'y': 11},
 {'x': '2022-03-28', 'y': 10},
 {'x': '2022-03-29', 'y': 7},
 {'x': '2022-03-30', 'y': 15},
 {'x': '2022-03-31', 'y': 9},
 {'x': '2022-04-01', 'y': 10},
 {'x': '2022-04-02', 'y': 11},
 {'x': '2022-04-03', 'y': 8},
 {'x': '2022-04-04', 'y': 10},
 {'x': '2022-04-05', 'y': 12},
 {'x': '2022-04-06', 'y': 13},
 {'x': '2022-04-07', 'y': 19},
 {'x': '2022-04-08', 'y': 13},
 {'x': '2022-04-09', 'y': 14},
 {'x': '2022-04-10', 'y': 7},
 {'x': '2022-04-11', 'y': 8},
 {'x': '2022-04-12', 'y': 16},
 {'x': '2022-04-13', 'y': 19},
 {'x': '2022-04-14', 'y': 18},
 {'x': '2022-04-15', 'y': 15},
 {'x': '2022-04-16', 'y': 12},
 {'x': '2022-04-17', 'y': 12},
 {'x': '2022-04-18', 'y': 17},
 {'x': '2022-04-19', 'y': 14},
 {'x': '2022-04-20', 'y': 12},
 {'x': '2022-04-21', 'y': 15},
 {'x': '2022-04-22', 'y': 17},
 {'x': '2022-04-23', 'y': 18}]

In [22]:
response

{'freqByDate': [{'x': '2022-03-26', 'y': 6},
  {'x': '2022-03-27', 'y': 11},
  {'x': '2022-03-28', 'y': 10},
  {'x': '2022-03-29', 'y': 7},
  {'x': '2022-03-30', 'y': 15},
  {'x': '2022-03-31', 'y': 9},
  {'x': '2022-04-01', 'y': 10},
  {'x': '2022-04-02', 'y': 11},
  {'x': '2022-04-03', 'y': 8},
  {'x': '2022-04-04', 'y': 10},
  {'x': '2022-04-05', 'y': 12},
  {'x': '2022-04-06', 'y': 13},
  {'x': '2022-04-07', 'y': 19},
  {'x': '2022-04-08', 'y': 13},
  {'x': '2022-04-09', 'y': 14},
  {'x': '2022-04-10', 'y': 7},
  {'x': '2022-04-11', 'y': 8},
  {'x': '2022-04-12', 'y': 16},
  {'x': '2022-04-13', 'y': 19},
  {'x': '2022-04-14', 'y': 18},
  {'x': '2022-04-15', 'y': 15},
  {'x': '2022-04-16', 'y': 12},
  {'x': '2022-04-17', 'y': 12},
  {'x': '2022-04-18', 'y': 17},
  {'x': '2022-04-19', 'y': 14},
  {'x': '2022-04-20', 'y': 12},
  {'x': '2022-04-21', 'y': 15},
  {'x': '2022-04-22', 'y': 17},
  {'x': '2022-04-23', 'y': 18}],
 'freqByCate': [368, 49, 3, 294, 0],
 'category': ['全部', '政治', 

## Save data to csv file

In [23]:
df_data = pd.DataFrame(list(response.items()),columns=['name','value'])

In [24]:
df_data

,name,value
0,freqByDate,"[{'x': '2022-03-26', 'y': 6}, {'x': '2022-03-2..."
1,freqByCate,"[368, 49, 3, 294, 0]"
2,category,"[全部, 政治, 產經, 生活, 社會]"
3,num_frequency,1306
4,num_occurrence,368


In [25]:
## 存成csv格式檔案
df_data.to_csv('chen_shih_chung_data.csv',sep=',', index=None)

### Alternative way: using zip

In [26]:
k=list(response.keys())
v=list(response.values())

In [27]:
#list(zip(k,v))

In [28]:
df_data = pd.DataFrame(list(zip(k,v)),columns=['name','value'])
df_data

,name,value
0,freqByDate,"[{'x': '2022-03-26', 'y': 6}, {'x': '2022-03-2..."
1,freqByCate,"[368, 49, 3, 294, 0]"
2,category,"[全部, 政治, 產經, 生活, 社會]"
3,num_frequency,1306
4,num_occurrence,368


## Read csv file and convert to dict format

In [29]:
df_data = pd.read_csv('chen_shih_chung_data.csv')

In [30]:
df_data 

,name,value
0,freqByDate,"[{'x': '2022-03-26', 'y': 6}, {'x': '2022-03-2..."
1,freqByCate,"[368, 49, 3, 294, 0]"
2,category,"['全部', '政治', '產經', '生活', '社會']"
3,num_frequency,1306
4,num_occurrence,368


In [31]:
# Convert to dictionary format
dict(list(df_data.values))

{'freqByDate': "[{'x': '2022-03-26', 'y': 6}, {'x': '2022-03-27', 'y': 11}, {'x': '2022-03-28', 'y': 10}, {'x': '2022-03-29', 'y': 7}, {'x': '2022-03-30', 'y': 15}, {'x': '2022-03-31', 'y': 9}, {'x': '2022-04-01', 'y': 10}, {'x': '2022-04-02', 'y': 11}, {'x': '2022-04-03', 'y': 8}, {'x': '2022-04-04', 'y': 10}, {'x': '2022-04-05', 'y': 12}, {'x': '2022-04-06', 'y': 13}, {'x': '2022-04-07', 'y': 19}, {'x': '2022-04-08', 'y': 13}, {'x': '2022-04-09', 'y': 14}, {'x': '2022-04-10', 'y': 7}, {'x': '2022-04-11', 'y': 8}, {'x': '2022-04-12', 'y': 16}, {'x': '2022-04-13', 'y': 19}, {'x': '2022-04-14', 'y': 18}, {'x': '2022-04-15', 'y': 15}, {'x': '2022-04-16', 'y': 12}, {'x': '2022-04-17', 'y': 12}, {'x': '2022-04-18', 'y': 17}, {'x': '2022-04-19', 'y': 14}, {'x': '2022-04-20', 'y': 12}, {'x': '2022-04-21', 'y': 15}, {'x': '2022-04-22', 'y': 17}, {'x': '2022-04-23', 'y': 18}]",
 'freqByCate': '[368, 49, 3, 294, 0]',
 'category': "['全部', '政治', '產經', '生活', '社會']",
 'num_frequency': '1306',
 'num

In [32]:
list(df_data.values)

[array(['freqByDate',
        "[{'x': '2022-03-26', 'y': 6}, {'x': '2022-03-27', 'y': 11}, {'x': '2022-03-28', 'y': 10}, {'x': '2022-03-29', 'y': 7}, {'x': '2022-03-30', 'y': 15}, {'x': '2022-03-31', 'y': 9}, {'x': '2022-04-01', 'y': 10}, {'x': '2022-04-02', 'y': 11}, {'x': '2022-04-03', 'y': 8}, {'x': '2022-04-04', 'y': 10}, {'x': '2022-04-05', 'y': 12}, {'x': '2022-04-06', 'y': 13}, {'x': '2022-04-07', 'y': 19}, {'x': '2022-04-08', 'y': 13}, {'x': '2022-04-09', 'y': 14}, {'x': '2022-04-10', 'y': 7}, {'x': '2022-04-11', 'y': 8}, {'x': '2022-04-12', 'y': 16}, {'x': '2022-04-13', 'y': 19}, {'x': '2022-04-14', 'y': 18}, {'x': '2022-04-15', 'y': 15}, {'x': '2022-04-16', 'y': 12}, {'x': '2022-04-17', 'y': 12}, {'x': '2022-04-18', 'y': 17}, {'x': '2022-04-19', 'y': 14}, {'x': '2022-04-20', 'y': 12}, {'x': '2022-04-21', 'y': 15}, {'x': '2022-04-22', 'y': 17}, {'x': '2022-04-23', 'y': 18}]"],
       dtype=object),
 array(['freqByCate', '[368, 49, 3, 294, 0]'], dtype=object),
 array(['category

In [33]:
response = dict(list(df_data.values))


In [34]:
response['num_occurrence']

'368'

In [35]:
type(response['num_occurrence'])


str

In [36]:
type(response['freqByCate'])


str

In [37]:
response['freqByCate'] = eval(response['freqByCate'])


In [38]:
response


{'freqByDate': "[{'x': '2022-03-26', 'y': 6}, {'x': '2022-03-27', 'y': 11}, {'x': '2022-03-28', 'y': 10}, {'x': '2022-03-29', 'y': 7}, {'x': '2022-03-30', 'y': 15}, {'x': '2022-03-31', 'y': 9}, {'x': '2022-04-01', 'y': 10}, {'x': '2022-04-02', 'y': 11}, {'x': '2022-04-03', 'y': 8}, {'x': '2022-04-04', 'y': 10}, {'x': '2022-04-05', 'y': 12}, {'x': '2022-04-06', 'y': 13}, {'x': '2022-04-07', 'y': 19}, {'x': '2022-04-08', 'y': 13}, {'x': '2022-04-09', 'y': 14}, {'x': '2022-04-10', 'y': 7}, {'x': '2022-04-11', 'y': 8}, {'x': '2022-04-12', 'y': 16}, {'x': '2022-04-13', 'y': 19}, {'x': '2022-04-14', 'y': 18}, {'x': '2022-04-15', 'y': 15}, {'x': '2022-04-16', 'y': 12}, {'x': '2022-04-17', 'y': 12}, {'x': '2022-04-18', 'y': 17}, {'x': '2022-04-19', 'y': 14}, {'x': '2022-04-20', 'y': 12}, {'x': '2022-04-21', 'y': 15}, {'x': '2022-04-22', 'y': 17}, {'x': '2022-04-23', 'y': 18}]",
 'freqByCate': [368, 49, 3, 294, 0],
 'category': "['全部', '政治', '產經', '生活', '社會']",
 'num_frequency': '1306',
 'num_o

### # How to convert a list into a dict?

In [39]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [40]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [41]:
dict([['one',[1,2,3]], ['two',2]])

{'one': [1, 2, 3], 'two': 2}

## All-in-one function

In [42]:
# Load Data 
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')

In [43]:


def process_data_all_in_one(query_keywords, weeks):


    cond='or'
    cate='全部'

    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    cate_freq, cate_occurrence, _, _ = count_keyword(df_query, query_keywords)
    freqByDate = get_keyword_occurrence_time_series(df_query)


    selectedCategories = ['全部', '政治', '生活', '社會']
    # selectedCategories = ['全部', '政治', '產經', '生活', '社會']
    # selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
    # selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

    freqByCate = [cate_occurrence[k] for k in selectedCategories]

    response =  {'freqByDate': freqByDate,
            'freqByCate': freqByCate,
            'category': selectedCategories,
            'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
            'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
            }

    

    return response

In [44]:
query_keywords = ['陳時中','時中阿伯'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [45]:
df_data = pd.DataFrame(list(data_response.items()),columns=['name','value'])
## 存成csv格式檔案
df_data.to_csv('chen_shih_chung_data.csv',sep=',', index=None)

In [46]:
df_data = pd.read_csv('chen_shih_chung_data.csv')

In [47]:
df_data 

,name,value
0,freqByDate,"[{'x': '2022-03-26', 'y': 6}, {'x': '2022-03-2..."
1,freqByCate,"[368, 49, 294, 0]"
2,category,"['全部', '政治', '生活', '社會']"
3,num_frequency,1306
4,num_occurrence,368


# This is your featured app

In [48]:
query_keywords = ['蔡英文','小英'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
#query_keywords = ['柯文哲','柯P','柯p'] #不要有子字串否則會重複計算 
#query_keywords = ['蘋果','Apple','ipad','iphone','imac'] #不要有子字串否則會重複計算 
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [49]:
data_response

{'freqByDate': [{'x': '2022-03-26', 'y': 8},
  {'x': '2022-03-27', 'y': 4},
  {'x': '2022-03-28', 'y': 5},
  {'x': '2022-03-29', 'y': 14},
  {'x': '2022-03-30', 'y': 13},
  {'x': '2022-03-31', 'y': 7},
  {'x': '2022-04-01', 'y': 14},
  {'x': '2022-04-02', 'y': 4},
  {'x': '2022-04-03', 'y': 6},
  {'x': '2022-04-04', 'y': 5},
  {'x': '2022-04-05', 'y': 6},
  {'x': '2022-04-06', 'y': 4},
  {'x': '2022-04-07', 'y': 13},
  {'x': '2022-04-08', 'y': 22},
  {'x': '2022-04-09', 'y': 9},
  {'x': '2022-04-10', 'y': 6},
  {'x': '2022-04-11', 'y': 10},
  {'x': '2022-04-12', 'y': 8},
  {'x': '2022-04-13', 'y': 6},
  {'x': '2022-04-14', 'y': 13},
  {'x': '2022-04-15', 'y': 10},
  {'x': '2022-04-16', 'y': 6},
  {'x': '2022-04-17', 'y': 3},
  {'x': '2022-04-18', 'y': 4},
  {'x': '2022-04-19', 'y': 7},
  {'x': '2022-04-20', 'y': 10},
  {'x': '2022-04-21', 'y': 3},
  {'x': '2022-04-22', 'y': 18},
  {'x': '2022-04-23', 'y': 7}],
 'freqByCate': [245, 184, 27, 8],
 'category': ['全部', '政治', '生活', '社會'],
 'n

In [50]:
query_keywords = ['疫情','新冠'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [51]:
data_response

{'freqByDate': [{'x': '2022-03-26', 'y': 46},
  {'x': '2022-03-27', 'y': 60},
  {'x': '2022-03-28', 'y': 99},
  {'x': '2022-03-29', 'y': 74},
  {'x': '2022-03-30', 'y': 104},
  {'x': '2022-03-31', 'y': 98},
  {'x': '2022-04-01', 'y': 75},
  {'x': '2022-04-02', 'y': 52},
  {'x': '2022-04-03', 'y': 57},
  {'x': '2022-04-04', 'y': 64},
  {'x': '2022-04-05', 'y': 70},
  {'x': '2022-04-06', 'y': 97},
  {'x': '2022-04-07', 'y': 119},
  {'x': '2022-04-08', 'y': 98},
  {'x': '2022-04-09', 'y': 74},
  {'x': '2022-04-10', 'y': 57},
  {'x': '2022-04-11', 'y': 103},
  {'x': '2022-04-12', 'y': 120},
  {'x': '2022-04-13', 'y': 150},
  {'x': '2022-04-14', 'y': 105},
  {'x': '2022-04-15', 'y': 94},
  {'x': '2022-04-16', 'y': 69},
  {'x': '2022-04-17', 'y': 71},
  {'x': '2022-04-18', 'y': 101},
  {'x': '2022-04-19', 'y': 122},
  {'x': '2022-04-20', 'y': 114},
  {'x': '2022-04-21', 'y': 116},
  {'x': '2022-04-22', 'y': 87},
  {'x': '2022-04-23', 'y': 56}],
 'freqByCate': [2552, 210, 1023, 38],
 'categor

# views.py in Django

In [52]:
from django.http import JsonResponse
from django.shortcuts import render
import pandas as pd

def load_data_scchen():
    # Read data from csv file
    df_data = pd.read_csv('app_scchen/dataset/chen_shih_chung_data.csv',sep=',')
    global response
    response = dict(list(df_data.values))
    del df_data

# load pk data
load_data_scchen()

def home(request):
    return render(request,'app_scchen/home.html', response)

print('app_scchen was loaded!')

FileNotFoundError: [Errno 2] No such file or directory: 'app_scchen/dataset/chen_shih_chung_data.csv'

        # Saving data with other file format

        ## Save to text file
        # Save data
        f = open('pk_politician.txt','w')
        f.write(str(data_pk))
        f.close()

        # Load data
        f = open('pk_politician.txt','r')
        data_pk = f.read()

        # dictionary format
        eval(data_pk)
        ## Save to json file
        import json

        # Save data
        with open('pk_politician.txt', 'w') as file:
            json.dump( data_pk, file)

        # Load data
        with open('pk_politician.txt', 'r') as file:
            data_pk = json.load(file)

        # dictionary format
        data_pk

        ## save to mongoDb (A famous NoneSQL)
        from pymongo import MongoClient

        # Save data
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH = database["pk_politicianTsaiHan"]   # SQL: Table Name

        table_KTH.drop()
        table_KTH.insert_one(data)


        # Load data
        from pymongo import MongoClient
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH  = database["pkData"]   # SQL: Table Name

        for x in table_KTH.find():
            print(x)

        list(table_KTH.find({}, {'_id': False}))
